In [ ]:
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import itertools
import random
import os
from pathlib import Path

# Import visualization libraries
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cv2
import seaborn as sns

# PyTorch & torchvision
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms, models

# Metrics
from sklearn.metrics import classification_report, accuracy_score

sns.set_style('darkgrid')

# %%
def seed_everything(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.use_deterministic_algorithms(True)

seed_everything()

In [ ]:
DATASET_DIR = "/kaggle/input/animals10/raw-img"  # adjust as needed
BATCH_SIZE = 32
TARGET_SIZE = (224, 224)
def convert_path_to_df(dataset):
    image_dir = Path(dataset)
    # Look for multiple image extensions
    filepaths = list(image_dir.glob('**/*.JPG')) + list(image_dir.glob('**/*.jpg')) + \
                list(image_dir.glob('**/*.jpeg')) + list(image_dir.glob('**/*.PNG'))
    # Assume the parent folder name is the label
    labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')
    image_df = pd.concat([filepaths, labels], axis=1)
    return image_df

image_df = convert_path_to_df(DATASET_DIR)

# Create a numeric label column
unique_labels = sorted(image_df['Label'].unique())


In [ ]:
image_df.head()

In [ ]:
print(unique_labels)

In [ ]:
label_mapping = {
    'cane': 'dog',
    'cavallo': 'horse',
    'elefante': 'elephant',
    'farfalla': 'butterfly',
    'gallina': 'chicken',
    'gatto': 'cat',
    'mucca': 'cow',
    'pecora': 'sheep',
    'ragno': 'spider',
    'scoiattolo': 'squirrel'
}
unique_labels = ['dog', 'horse', 'elephant', 'butterfly', 'chicken', 'cat', 'cow', 'sheep', 'spider', 'squirrel']
# Assuming your DataFrame is named 'df' and the column with labels is 'Label'
image_df['Label'] = image_df['Label'].map(label_mapping)

In [ ]:
label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}
idx_to_label = {idx: label for label, idx in label_to_idx.items()}
image_df['Label_idx'] = image_df['Label'].map(label_to_idx)

In [ ]:
import PIL
from PIL import UnidentifiedImageError, Image

for img_p in Path(DATASET_DIR).rglob("*.jpg"):
    try:
        img = PIL.Image.open(img_p)
    except UnidentifiedImageError:
        print(f'Could not identify image: {img_p}')

# %%
# Get the value counts for each label and plot distribution
label_counts = image_df['Label'].value_counts()

fig, ax = plt.subplots(figsize=(20, 6))
sns.barplot(x=label_counts.index, y=label_counts.values, alpha=0.8, palette='pastel', ax=ax)
ax.set_title('Distribution of Labels in Image Dataset', fontsize=16)
ax.set_xlabel('Label', fontsize=14)
ax.set_ylabel('Count', fontsize=14)
ax.set_xticklabels(label_counts.index, rotation=45)
fig.suptitle('Image Dataset Label Distribution', fontsize=20)
fig.subplots_adjust(top=0.85)
plt.show()


In [ ]:
# Display 16 picture of the dataset with their labels
random_index = np.random.randint(0, len(image_df), 16)
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(10, 10),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(image_df.Filepath[random_index[i]]))
    ax.set_title(image_df.Label[random_index[i]])
plt.tight_layout()
plt.show()

In [ ]:
def compute_ela_cv(path, quality):
    temp_filename = 'temp_file_name.jpeg'
    SCALE = 15
    orig_img = cv2.imread(path)
    orig_img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
    
    cv2.imwrite(temp_filename, orig_img, [cv2.IMWRITE_JPEG_QUALITY, quality])

    # read compressed image
    compressed_img = cv2.imread(temp_filename)

    # get absolute difference between img1 and img2 and multiply by scale
    diff = SCALE * cv2.absdiff(orig_img, compressed_img)
    return diff


def random_sample(path, extension=None):
    if extension:
        items = Path(path).glob(f'*.{extension}')
    else:
        items = Path(path).glob(f'*')
        
    items = list(items)
        
    p = random.choice(items)
    return p.as_posix()

In [ ]:
# View random sample from the dataset
p = random_sample('/kaggle/input/animals10/raw-img/cane')
orig = cv2.imread(p)
orig = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB) / 255.0
init_val = 100
columns = 3
rows = 3

fig=plt.figure(figsize=(15, 10))
for i in range(1, columns*rows +1):
    quality=init_val - (i-1) * 8
    img = compute_ela_cv(path=p, quality=quality)
    if i == 1:
        img = orig.copy()
    ax = fig.add_subplot(rows, columns, i) 
    ax.title.set_text(f'q: {quality}')
    plt.imshow(img)
plt.show()

In [ ]:
# Separate in train and test data
train_df, test_df = train_test_split(image_df, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
class AnimalDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.loc[idx, 'Filepath']
        image = Image.open(img_path).convert('RGB')
        label = int(self.dataframe.loc[idx, 'Label_idx'])
        if self.transform:
            image = self.transform(image)
        return image, label

train_transform = transforms.Compose([
    transforms.Resize(TARGET_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),  # degrees
    transforms.RandomAffine(degrees=0, scale=(0.9, 1.1)),  # simulate zoom\n    transforms.ColorJitter(contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize(TARGET_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
train_dataset = AnimalDataset(train_df, transform=train_transform)
val_dataset = AnimalDataset(test_df, transform=test_transform)  # using test_df as validation for now

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
model = models.efficientnet_v2_m(weights='DEFAULT')

# Freeze the feature extractor
for param in model.parameters():
    param.requires_grad = False

In [ ]:

in_features = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Linear(in_features, 128),
    nn.ReLU(),
    nn.BatchNorm1d(128),
    nn.Dropout(0.45),
    nn.Linear(128, 256),
    nn.ReLU(),
    nn.BatchNorm1d(256),
    nn.Dropout(0.45),
    nn.Linear(256, len(unique_labels))  # number of classes
)

model = model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
DEVICE = next(model.parameters()).device

# %%
# Loss, Optimizer, and Learning Rate Scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=3, min_lr=1e-6)

In [ ]:
num_epochs = 25
best_val_loss = np.inf

patience = 5
epochs_without_improvement = 0

train_history = {'loss': [], 'acc': []}
val_history = {'loss': [], 'acc': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    
    train_loss = running_loss / total
    train_acc = 100 * correct / total
    train_history['loss'].append(train_loss)
    train_history['acc'].append(train_acc)
    
    model.eval()
    val_running_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)
    val_loss = val_running_loss / val_total
    val_acc = 100 * val_correct / val_total
    val_history['loss'].append(val_loss)
    val_history['acc'].append(val_acc)
    
    scheduler.step(val_loss)
    
    print(f'Epoch {epoch+1}/{num_epochs} -- Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0
        torch.save(model.state_dict(), 'animals_classification_model_checkpoint.pth')
    else:
        epochs_without_improvement += 1

    if epochs_without_improvement >= patience:
        print("Early stopping triggered.")
        break


# %%
# Load the best model for evaluation
model.load_state_dict(torch.load('animals_classification_model_checkpoint.pth'))
model.eval()

In [ ]:

all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(DEVICE)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_loss = np.mean([criterion(model(images.to(DEVICE)), labels.to(DEVICE)).item()
                      for images, labels in val_loader])
print(f"Test Loss: {test_loss:.5f}")
print(f"Test Accuracy: {accuracy_score(all_labels, all_preds)*100:.2f}%")

# %%
# Plot Training History
epochs_range = range(len(train_history['loss']))
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1.plot(epochs_range, train_history['acc'], 'b', label='Training Accuracy')
ax1.plot(epochs_range, val_history['acc'], 'r', label='Validation Accuracy')
ax1.set_title('Training and Validation Accuracy')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Accuracy')
ax1.legend()

ax2.plot(epochs_range, train_history['loss'], 'b', label='Training Loss')
ax2.plot(epochs_range, val_history['loss'], 'r', label='Validation Loss')
ax2.set_title('Training and Validation Loss')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Loss')
ax2.legend()

fig.suptitle('Training and Validation Metrics', fontsize=16)
plt.show()

In [ ]:
# Predict the labels for the validation set
model.eval()
all_val_preds = []
with torch.no_grad():
    for images, _ in val_loader:
        images = images.to(DEVICE)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_val_preds.extend(preds.cpu().numpy())

# Map numeric labels back to class names
pred_labels = [idx_to_label[p] for p in all_val_preds]
print(f'The first 5 predictions: {pred_labels[:5]}')

# %%
# Display 15 random images from the test set with true and predicted labels
random_idx = np.random.randint(0, len(test_df), 15)
fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(25, 15), subplot_kw={'xticks': [], 'yticks': []})

# Get predictions for the entire test set
test_loader_full = DataLoader(AnimalDataset(test_df, transform=test_transform), batch_size=BATCH_SIZE, shuffle=False)
all_test_preds = []
with torch.no_grad():
    for images, _ in test_loader_full:
        images = images.to(DEVICE)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_test_preds.extend(preds.cpu().numpy())

for i, ax in enumerate(axes.flat):
    idx = random_idx[i]
    img = plt.imread(test_df.Filepath.iloc[idx])
    true_label = test_df.Label.iloc[idx]
    pred_label = idx_to_label[all_test_preds[idx]]
    color = "green" if true_label == pred_label else "red"
    ax.imshow(img)
    ax.set_title(f"True: {true_label}\nPredicted: {pred_label}", color=color)
plt.tight_layout()
plt.show()

# %%
# Compute and display classification report
y_true = list(test_df.Label)
y_pred = [idx_to_label[p] for p in all_test_preds]
print(classification_report(y_true, y_pred))



In [ ]:
report = classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
print(df_report)